In [49]:
# Imports
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.cm import get_cmap
from matplotlib.colors import ListedColormap
from matplotlib.colorbar import Colorbar
%matplotlib inline
import metpy.calc as mpcalc
from metpy.units import units
import skimage.measure
import skimage.morphology
import scipy.ndimage
import warnings

# Filter warnings
warnings.filterwarnings('ignore')

In [14]:
# Define functions
def open_ds(bubble, wind, hres, vres):
    filepath = (f'/chinook2/stluthi/cm1_output/{bubble}_bubble/{wind}/{hres}dx_{vres}/cm1out_{hres}dx_{vres}.nc')
    return xr.open_dataset(filepath)

In [15]:
# Get data
cm1_1km = open_ds('single', 'no_wind', '1km', '50vlvls')

In [100]:
# For loop to loop through times in dataset for LOW LEVEL VERTICAL VELOCITY
for hour in range(1, 17, 1):
    
    # Get variables
    vertVelo = cm1_1km['w']
    zf = cm1_1km['zf']
    
    # Slice vertical velocity for 1-4km AGL
    vertVeloSlice = vertVelo[hour, 11:20, :, :]
    
    # Create composite image of vertical velocity
    vertVeloComp = vertVeloSlice.max(dim='zf')
    
    # Set threshold for vertical velocity
    threshVertVelo = vertVeloComp > 9
    
    # Create buffered vertical velocity
    res = '1km'
    buffer = '5km'
    buffer_px = int((units(buffer) / units(res)).m)
    bufferedVV = skimage.morphology.binary_dilation(threshVertVelo.values, footprint=skimage.morphology.disk(buffer_px))
    
    # Create initial labeled image and remove small objects
    initLabeledImage, _ = scipy.ndimage.label(bufferedVV, np.ones((3,3), dtype=int))
    skimage.morphology.remove_small_objects(initLabeledImage, min_size=buffer_px * 3, connectivity=2)
    
    # Create list for vertical velocity subsets
    subsets_VV = []
    
    # For loop to fill subsets_VV
    for region in skimage.measure.regionprops(initLabeledImage):
        subsets_VV.append(vertVeloComp.isel(yh=slice(region.bbox[0], region.bbox[2]),
                                            xh=slice(region.bbox[1], region.bbox[3])))
        #plt.imshow(initLabeledImage)
    
    # Create lists for desire properties
    contour_num = 0
    contour_areaTotal = 0
    
    # Loop through subsets_VV and calculate properties
    for subset_VV in subsets_VV:
        labeled_image, num_contours = skimage.measure.label(subset_VV.values >= 10, return_num=True)
        contour_regions = skimage.measure.regionprops(labeled_image, intensity_image=subset_VV.values)
        contour_areas = [region.area for region in contour_regions]
        contour_intensities = [region.intensity_mean for region in contour_regions]
        contour_num = contour_num + num_contours
        contour_area = sum(contour_areas)
        contour_areaTotal = contour_areaTotal + contour_area
        print('Hour: ', (hour * 0.5) , '|| Number of Contours: ', num_contours)
        print(contour_intensities)

Hour:  0.5 || Number of Contours:  1
[14.111469]
Hour:  1.0 || Number of Contours:  4
[11.913318, 11.913318, 11.913318, 11.913318]
Hour:  1.5 || Number of Contours:  16
[10.363264, 10.363264, 13.029282, 13.029282, 13.029282, 13.029282, 10.363264, 10.363264, 10.363264, 13.029282, 13.029282, 10.363264, 13.029282, 13.029282, 10.363264, 10.363264]
Hour:  2.0 || Number of Contours:  28
[14.771059, 14.771059, 10.395899, 10.395899, 15.5655775, 15.5655775, 13.773045, 13.773045, 10.395899, 10.395899, 14.771059, 15.5655775, 15.5655775, 14.771059, 14.771059, 15.5655775, 15.5655775, 14.771059, 10.395899, 10.395899, 13.773045, 13.773045, 15.5655775, 15.5655775, 10.395899, 10.395899, 14.771059, 14.771059]
Hour:  2.5 || Number of Contours:  1
[11.669203]
Hour:  2.5 || Number of Contours:  1
[11.669203]
Hour:  2.5 || Number of Contours:  5
[15.041505, 11.399957, 15.041505, 11.282874, 11.282874]
Hour:  2.5 || Number of Contours:  5
[11.282874, 15.041505, 11.399957, 15.041505, 11.282874]
Hour:  2.5 || N